In [10]:
%%capture
!pip install -q dagshub mlflow langchain langchain-openai

In [ ]:
#@markdown **You need to sign up for [DagsHub](https://dagshub.com/user/sign_up) , then enter the name of the repository you'd like to create, and your username and email.**

#@markdown Enter the repository name for the project:
REPO_NAME= "komrag" #@param {type:"string"}

#@markdown Enter the username of your DAGsHub account:
USER_NAME = "OsipovStas" #@param {type:"string"}

#@markdown Enter the email for your DAGsHub account:
EMAIL = "stasstels@gmail.com" #@param {type:"string"}

#@markdown ---

import IPython

In [ ]:
!dagshub login

import dagshub
TOKEN = dagshub.auth.get_token()

                                ❗❗❗ AUTHORIZATION REQUIRED ❗❗❗                                


Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=93bfb02a-fa30-465e-a801-d4e010c9cd5f&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=7578421f4192f84f4147c6c8972f33200102e679325d34a8ced93e65a66e5789


⠏ Waiting for authorization
✅ OAuth token added


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=9e460156-51c3-403b-8d36-dec5e4803790&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=e4003743f906c8f3adea74440c439c446d3b5c928422b54793a19c40382e7c0e




Output()

Accessing as OsipovStas

In [ ]:
from google.colab import userdata
DH_TOKEN = userdata.get('LLM_MATH_COMP_DH_TOKEN')


In [ ]:
import mlflow
import dagshub
import os
#DH_TOKEN = os.environ['LLM_MATH_COMP_DH_TOKEN']
username = f'{USER_NAME}'  # Replace with your DagsHub username
repository = f'{REPO_NAME}'  # Replace with your repository name
token = f'{DH_TOKEN}'  # Replace with your DagsHub token
os.environ['MLFLOW_TRACKING_USERNAME'] = username
os.environ['MLFLOW_TRACKING_PASSWORD'] = token
dagshub.auth.add_app_token(token)
dagshub.init(repo_name=REPO_NAME, repo_owner=USER_NAME)
mlflow.set_tracking_uri(f"https://dagshub.com/{USER_NAME}/{REPO_NAME}.mlflow")

Initialized MLflow to track repo "OsipovStas/komrag"

Repository OsipovStas/komrag initialized!

In [ ]:
from dagshub.notebook import save_notebook

save_notebook(repo=f"{USER_NAME}/{REPO_NAME}", path="./notebooks/exploratory.ipynb")

In [5]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("OsipovStas/komrag")

# Upload file
s3.upload_file(
    Bucket="komrag",  # name of the repo
    Filename="./life_begin.pdf",  # local path of file to upload
    Key="life_begin.pdf",  # remote path where to upload the file
)

Client created. Use the name of the repo (komrag) as the name of the bucket

In [6]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("OsipovStas/komrag")


s3.download_file(
    Bucket="komrag",  # name of the repo
    Key="life_begin.pdf",  #  remote path from where to download the file
    Filename="life_begin_copy.pdf",  # local path where to download the file
)

Client created. Use the name of the repo (komrag) as the name of the bucket

# LLM Configuration

In [9]:
import os

os.environ['AZURE_OPENAI_ENDPOINT'] = userdata.get("AZURE_OPENAI_ENDPOINT")
os.environ['AZURE_OPENAI_API_KEY'] = userdata.get('AZURE_KEY')
os.environ['OPENAI_API_VERSION'] = '2024-02-01'

In [12]:
model="gpt-4o-mini-2024-07-18"

In [19]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model=model, temperature=0)

In [22]:
llm.invoke("Hello")

KeyboardInterrupt: 